<a href="https://colab.research.google.com/github/harnalashok/hadoop/blob/main/spark_pipelining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 04th April, 2023
# Myfolder: github/hadoop
# Objectives:
#             i)  Install pyspark on colab
#             ii) Install koalas on colab
#                 (version installed is spark-3.3.2)
#
# Java 8 install: https://stackoverflow.com/a/58191107
# Hadoop install: https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html
# Spark install:  https://stackoverflow.com/a/64183749
#                 https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

# Spark Reference API
a. [Quickstart](https://spark.apache.org/docs/latest/quick-start.html) <br>
b. Dataframe [APIs list](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html) at a glance<br>
c. Pandas [API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/index.html) on spark <br>
c. ALso look at useful [this source code](https://spark.apache.org/docs/latest/api/python/_modules/pyspark/sql/functions.html) of functions that has examples<br>



# A. Full spark install
Installs `pyspark (spark-3.3.2-bin-hadoop3)` 

### 1.0 Libraries

In [1]:
# 1.0 How to set environment variable
import os  
import time  

## 2.0 Define some functions

#### ssh_install()

In [2]:
# 2.0 Function to install ssh client and sshd (Server)
def ssh_install():
  print("\n--1. Download and install ssh server----\n")
  ! sudo apt-get remove openssh-client openssh-server
  ! sudo apt install openssh-client openssh-server
  
  print("\n--2. Restart ssh server----\n")
  ! service ssh restart

#### Java install

In [3]:
# 3.0 Function to download and install java 8
def install_java():
  ! rm -rf /usr/java

  print("\n--Download and install Java 8----\n")
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null        # install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     # set environment variable

  !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
  !update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
  
  !mkdir -p /usr/java
  ! ln -s "/usr/lib/jvm/java-8-openjdk-amd64"  "/usr/java"
  ! mv "/usr/java/java-8-openjdk-amd64"  "/usr/java/latest"
  
  !java -version       #check java version
  !javac -version

#### setup ssh passphrase

In [4]:
# 6.0 Function tp setup ssh passphrase
def set_keys():
  print("\n---22. Generate SSH keys----\n")
  ! cd ~ ; pwd 
  ! cd ~ ; ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
  ! cd ~ ; cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
  ! cd ~ ; chmod 0600 ~/.ssh/authorized_keys


#### Set environment

In [5]:
# 7.0 Function to set up environmental variables
def set_env():
  print("\n---23. Set Environment variables----\n")
  # 'export' command does not work in colab
  # https://stackoverflow.com/a/57240319
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"   
  

#### function to install prerequisites
java and ssh<br>


In [6]:
# 8.0 Function to call all functions
def install_components():
  print("\n--Install java----\n")
  ssh_install()
  install_java()  
  #set_keys()
  set_env()


## 3.0 Install components
Start downloading, install and configure. Takes around 2 minutes<br>
Your <u>input *'y'* is required </u>at one place while overwriting earlier ssh keys

In [7]:
# 9.0 Start installation
start = time.time()
install_components()
end = time.time()
print("\n---Time taken----\n")
print((end- start)/60)


--Install java----


--1. Download and install ssh server----

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'openssh-server' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  libboost-atomic-dev libboost-atomic1.71-dev libboost-atomic1.71.0
  libboost-chrono-dev libboost-chrono1.71-dev libboost-chrono1.71.0
  libboost-container-dev libboost-container1.71-dev libboost-container1.71.0
  libboost-context-dev libboost-context1.71-dev libboost-context1.71.0
  libboost-coroutine-dev libboost-coroutine1.71-dev libboost-coroutine1.71.0
  libboost-date-time-dev libboost-date-time1.71-dev libboost-date-time1.71.0
  libboost-exception-dev libboost-exception1.71-dev libboost-fiber-dev
  libboost-fiber1.71-dev libboost-fiber1.71.0 libboost-filesystem-dev
  libboost-filesystem1.71-dev libboost-filesystem1.71.0 libboost-graph-dev
  libboost-graph-parallel-dev libboost-graph-par

## 4.0 Install spark


### Define functions

`findspark`: PySpark isn't on `sys.path` by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding `pyspark` to `sys.path` at runtime. `findspark` does the latter.

In [8]:
# 1.0 Function to download and unzip spark
def spark_koalas_install():
  print("\n--1.1 Install findspark----\n")
  !pip install -q findspark

  #print("\n--1.2 Install databricks Koalas----\n")
  #!pip install koalas
  
  # This download link NEEDS TO BE CHECKED AGAIN
  print("\n--1.3 Download Apache tar.gz----\n")
            
  ! wget -c https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz 

  print("\n--1.4 Transfer downloaded content and unzip tar.gz----\n")
  !  mv /content/spark*   /opt/
  ! tar -xzf /opt/spark-3.3.2-bin-hadoop3.tgz  --directory /opt/

  print("\n--1.5 Check folder for files----\n")
  ! ls -la /opt


In [9]:
# 1.1 Function to set environment
def set_spark_env():
  print("\n---2. Set Environment variables----\n")
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre" 
  os.environ["SPARK_HOME"] = "/opt/spark-3.3.2-bin-hadoop3" 
  os.environ["SPARK_CONF_DIR"] = "/opt/spark-3.3.2-bin-hadoop3/conf"     
  os.environ["LD_LIBRARY_PATH"] += ":/opt/spark-3.3.2-bin-hadoop3/lib/native"
  os.environ["PATH"] += ":/opt/spark-3.3.2-bin-hadoop3/bin:/opt/spark-3.3.2-bin-hadoop3/sbin"
  print("\n---2.1. Check Environment variables----\n")
  # Check
  ! echo $PATH
  ! echo $LD_LIBRARY_PATH

In [10]:
# 1.2 Function to configure spark 
def spark_conf():
  print("\n---3. Configure spark to access hadoop----\n")
  !mv /opt/spark-3.3.2-bin-hadoop3/conf/spark-env.sh.template  /opt/spark-3.3.2-bin-hadoop3/conf/spark-env.sh
  #!echo "HADOOP_CONF_DIR=/opt/hadoop-3.2.2/etc/hadoop/" >> /opt/spark-3.1.2-bin-hadoop3.2/conf/spark-env.sh
  print("\n---3.1 Check ----\n")
  #!cat /opt/spark-3.1.1-bin-hadoop3.2/conf/spark-env.sh

### Install spark


In [11]:
# 2.0 Call all the three functions
def install_spark():
  spark_koalas_install()
  set_spark_env()
  spark_conf()


In [12]:
# 2.1 
install_spark()


--1.1 Install findspark----


--1.3 Download Apache tar.gz----

--2023-04-04 03:31:37--  https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299360284 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.2-bin-hadoop3.tgz’

spark-3.3.2-bin-had 100%[===================>] 285.49M   197MB/s    in 1.5s    

2023-04-04 03:31:39 (197 MB/s) - ‘spark-3.3.2-bin-hadoop3.tgz’ saved [299360284/299360284]


--1.4 Transfer downloaded content and unzip tar.gz----


--1.5 Check folder for files----

total 292368
drwxr-xr-x  1 root root      4096 Apr  4 03:31 .
drwxr-xr-x  1 root root      4096 Apr  4 03:30 ..
drwxr-xr-x  1 root root      4096 Mar 31 13:52 google
drwxr-xr-x  1 root root      4096 Mar 31 13:28 nvidia
drwxr-xr-x 13  501 1000      4096 Feb 10 20:40 spark-3.3.

# B. Call libraries
We do not import `pandas` but `pyspark.pandas` as `ps`.

In [16]:
# 3.0 Just call some libraries to test
import numpy as np
import os
import time 

# 3.1 Get spark in sys.path
import findspark
findspark.init()

# 3.2 Call other spark libraries
#     Just to test
import pyspark.pandas as ps
from pyspark.sql import SparkSession
#import databricks.koalas as ks

In [17]:
# 3.3
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [18]:
# 3.4 Increase cell width to display wide columnar output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# C. Build spark session
You can modify spark driver/executor memory here<br>
`SparkSession` name is `spark`<br>



## Modifying spark configuraion
Increase driver and executor memory

In [19]:
# 4.0 Check template file
! cat /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf.template

#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Default system properties included when running spark-submit.
# This is useful for setting default environmental settings.

# Example:
# spark.master                     spark://master:7077
# spark.eventLog.enable

In [20]:
# 4.1 Create spark-defaults.conf 
! cp /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf.template  /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf


In [21]:
# 4.2 Amend properties
! echo "spark.driver.memory 6g" >> /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf
! echo "spark.executor.memory 3g" >> /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf

In [22]:
# 4.3 Check now
! cat /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf

#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Default system properties included when running spark-submit.
# This is useful for setting default environmental settings.

# Example:
# spark.master                     spark://master:7077
# spark.eventLog.enable

## Stop and start SparkSession

In [23]:
# 5.0 Build spark session:
# Stop spark, if started
#spark.stop()
# 5.1 Now start spark
spark = SparkSession. \
                    builder. \
                    master("local[*]"). \
                    appName("myexpt"). \
                    getOrCreate()

In [24]:
sc = spark.sparkContext
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.executor.memory', '3g'),
 ('spark.app.id', 'local-1680579121550'),
 ('spark.app.name', 'myexpt'),
 ('spark.app.startTime', '1680579119059'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '34447'),
 ('spark.driver.

In [25]:
# 5.2
print(spark.sparkContext._conf.getAll())

[('spark.driver.extraJavaOptions', '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'), ('spark.executor.memory', '3g'), ('spark.app.id', 'local-1680579121550'), ('spark.app.name', 'myexpt'), ('spark.app.startTime', '1680579119059'), ('spark.executor.id', 'driver'), ('spark.driver.port', '34447'), ('spark.driver.host', 'd

# D. Test spark
Use existing *spark* session to test if spark is working


In [62]:
# 6.0 pyspark pandas DataFrame
pdf = ps.DataFrame({
        'x1': ['a','a','b','b','', 'c', 'd','d'],
        'x2': ['apple','', 'orange','orange', 'peach','','apple','orange'],
        'x3': [1, 1, 2, 2, 2, 4, 1, 2],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5, 3.0, 2.0],
        'y1': [1, 0, 1, 0, 0, 1, 1, 0],
        'y2': ['yes', 'no', 'no','','', 'yes','', 'yes']
    })

# 6.1
pdf

,x1,x2,x3,x4,y1,y2
0,a,apple,1,2.4,1,yes
1,a,,1,2.5,0,no
2,b,orange,2,3.5,1,no
3,b,orange,2,1.4,0,
4,,peach,2,2.1,0,
5,c,,4,1.5,1,yes
6,d,apple,1,3.0,1,
7,d,orange,2,2.0,0,yes


In [63]:
# 6.2 Transform to Spark DataFrame
#     and print
df = pdf.to_spark()
df.show()

/opt/spark-3.3.2-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|      |  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|   |
|   | peach|  2|2.1|  0|   |
|  c|      |  4|1.5|  1|yes|
|  d| apple|  1|3.0|  1|   |
|  d|orange|  2|2.0|  0|yes|
+---+------+---+---+---+---+



In [64]:
############

# E. Some useful functions
Execute only if you need. Else, forget it

In [ ]:
# 7.0 Per column how many null values:
from pyspark.sql.functions import isnan, when, count, col
def null_values(data):
  data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

In [ ]:
# 8.0 Finding mode of a column
# Refer StackOverflow: https://stackoverflow.com/a/58279672
def mode(df,col):
  df.groupby("col").count().orderBy("count", ascending=False).first()[0]

# 9.0 Find mode of all columns
def mode_cols(df):
  [[i,df.groupby(i).count().orderBy("count", ascending=False).first()[0]] for i in df.columns]  

In [ ]:
# 10.0 Value counts
def value_counts(df):
    for colm in df.columns:
        df.groupby(colm).count().show()

In [ ]:
# 11.0 Map a string to another
# See here: https://stackoverflow.com/a/55026324/3282777
# Code not tested
# Map a column 'colname' in dataframe 'df'
# as follows:
#
# map_dict= {
#            'A': '1',
#            'B': '2'
#           }

def mapping(df,map_dict, colname):
  df2 = df.replace(to_replace=map_dict, subset=['yourColName'])

# F. Your experiments
SparkSession is <i>'spark'</i>. Call all needed libraries. Begin with mounting gdrive.

In [65]:
# 1.0 Mount gdrive
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [111]:
# 2.0 Call ML libraries:

from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [67]:
# 2.1 List path to folder containg csv file. 
#      Note '/' at the end:

pathToFolder = "/gdrive/MyDrive/Colab_data_files/bankmarketing/"

In [100]:
# 2.2 Read data:

df = ps.read_csv(
                 pathToFolder + "bank-full.csv",
                 sep= ";"
                 )


/opt/spark-3.3.2-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [101]:
# 2.3 Your cat cols and num cols:

catCols = df.select_dtypes(include = ['object']).columns
numCols = df.select_dtypes(include = ['int32']).columns

catCols = catCols.to_list()
print("\n============\n")
numCols = numCols.to_list()

In [ ]:
# 2.4 What names catCols will have after StringIndexing and OHE:

outCols = [i + "_index" for i in list(catCols)[:-1]]  # Exclude y
outCols
oheCols = [i + "_vec" for i in list(catCols)[:-1]]
oheCols

vaCols = oheCols.copy()
vaCols.extend(numCols)
vaCols

In [103]:
# 3.0 Instantiate StringIndexer for multiple columns:

indexer = StringIndexer(
                        inputCols=catCols[:-1],   # Exclude y
                        outputCols=outCols
                        )


encoder = OneHotEncoder(inputCols=outCols,
                        outputCols=oheCols)


yindex = StringIndexer(inputCol = 'y',
                       outputCol = 'y_index')


vectorAssembler = VectorAssembler(
                                   inputCols = vaCols,
                                   outputCol = 'features'
                                  )

rf = RandomForestClassifier(
                             labelCol = 'y_index',
                             numTrees=100
                            )

In [104]:
# 3.1 Split spark_df into train/test:

splits = df.to_spark().randomSplit([0.7, 0.3])
type(splits)   # list


/opt/spark-3.3.2-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


list

In [105]:
# 3.2
train_df = splits[0]
test_df = splits[1]

In [106]:
# 4.0 Pipeline now:
pipeline = Pipeline(stages= [indexer, encoder, vectorAssembler, yindex, rf ])

In [107]:
# 4.1 fit pipeline:
model_p = pipeline.fit(train_df)

In [108]:
# 5.0 Make predictions:

test_df = model_p.transform(test_df)

In [109]:
# 5.1:

test_df.show(2)

+---+-------+-------+---------+------------------+----------------+--------------+---------------+---------------+---------------------+----------------------+---------------+-----------------------------------+-----+---------------------------+-----------------------------+---+---------+-------------+---------------+------------------------+--------------------+---------------------+---------------------+----------------------------+-----------------------------------+---------------+-------------+-------------+----------------------+------------------+-------------------+-------------------+--------------------------+---------------------------------+--------------------+-------+--------------------+--------------------+----------+
|age|    job|marital|education|hascreditindefault|avgyearlybalance|hashousingloan|haspersonalloan|contactCommType|lastContactdayOfMonth|lastContactmonthOfYear|contactDuration|noOfContactsPerformedDuringcampaign|pdays|noOfContprevious_tocampaign|outcomeOfPr

In [112]:
# 5.2 Evaluate:

evaluator = BinaryClassificationEvaluator(labelCol = 'y_index', metricName = 'areaUnderROC' )

In [113]:
# 5.2.1
evaluator.evaluate(test_df)   # 0.8938265864071989

0.8914511459642704

In [ ]:
################ I am done #################3